In [3]:
# import modules
import os
import sys
import pandas as pd
import numpy as np
import datetime
import csv
import json
import sqlite3
from pathlib import Path
from sklearn.manifold import TSNE
from sklearn.datasets import fetch_20newsgroups
import matplotlib.pyplot as plt
import gensim, logging

## Parameters you can change

# Abs path to settings file
#root = "C:/DS/Github/MusicRecommendation"  # BA, Windows
root = "/home/badrul/Documents/github/MusicRecommendation" # BA, Linux

## Import the codebase module
fPath = root + "/1_codemodule"
if fPath not in sys.path: sys.path.append(fPath)

import coreCode as cb
import lastfmCode as fm

## Finish setting up
os.chdir(root)

settingsDict =  cb.loadSettings()

dbPath = root + settingsDict['mainDbPath']
fmSimilarDbPath = root + settingsDict['fmSimilarDbPath']
fmTagsDbPath = root + settingsDict['fmTagsDbPath']
trackMetaDbPath = root + settingsDict['trackmetadata']

def GenerateWord2VecModel(trainingSetDbPath,threshold,iterations=50):
    s = fm.getSimilariityAll(fmSimilarDbPath,threshold)

    #s=getDummyData()

    # train word2vec
    model = gensim.models.Word2Vec(s, min_count=0, iter=iterations,size=100)
    model.save(root + '/3_data/song2vec/model')

def getDummyData():
    return [['Tiger', 'Zebra'], ['Tiger', 'Lion', 'Cheetah'],
     ['Orangutan', 'Bonobo', 'Orangutan', 'Chimpanzee'],
     ['Dog', 'Cat', 'Mouse'], ['Tiger', 'Rhino'],
     ['House', 'Car'], ['Antelope', 'Gazelle'],
     ['Zebra', 'Horse'], ['Tiger', 'Lion', 'Leopard'],
     ['Cat', 'Mouse'], ['Mouse', 'Hampster', 'Gerbil'],
     ['Rhino', 'Zebra'], ['Zebra', 'Antelope'],
     ['Tiger', 'Lion'], ['Lion', 'Tiger', 'Giraffe'],
     ['Leopard', 'Lion'], ['Leopard', 'Tiger', 'Lion'],
     ['Tiger', 'Lion'], ['Tiger', 'Lion'],
     ['Car', 'Van'], ['Car', 'Lorry'],
     ['Car', 'Van'], ['Car', 'Lorry'],
     ['Car', 'Van'], ['Car', 'Lorry']
     ]

def generateTSNE(V,vocab):
    # Generate tsne graph
    tsne = TSNE(n_components=2)  # 2 dimensions so we can plot it on an xy chart

    X_tsne = tsne.fit_transform(V)

    # Join the x,y values with the labels
    df = pd.concat([pd.DataFrame(X_tsne),
                    pd.Series(vocab)],
                   axis=1)

    df.columns = ['x', 'y', 'label']

    # Create chart
    fig = plt.figure()  # Create a canvas for the chart
    ax = fig.add_subplot(1, 1, 1)  # Create a 1x1 grid in the the 1st subplot
    ax.scatter(df['x'], df['y'])

    stepSize = 1
    for i, txt in enumerate(df['label']):
        if i % stepSize == 0:
            ax.annotate(txt, (df['x'].iloc[i], df['y'].iloc[i]))

    plt.show()
    
def w2v_model_accuracy(model,testSet = []):
    
    accuracy = 0
    for s in testSet:
        accuracy += model.score(testSet)
    
    print (accuracy/len[testSet])
    
    #sum_corr = len(accuracy[-1]['correct'])
    #sum_incorr = len(accuracy[-1]['incorrect'])
    #total = sum_corr + sum_incorr
    #percent = lambda a: a / total * 100
    
    #print('Total sentences: {}, Correct: {:.2f}%, Incorrect: {:.2f}%'.format(total, percent(sum_corr), percent(sum_incorr)))

<h4>Train Word2Vec model </h4>

In [8]:
# Load data from database

# This command line was used to import from the similarities.db
# sqlite3 "lastfmSimilar.db" ".dump" | sqlite3 "inputDB.db"

# Generate Gensim Model
#GenerateWord2VecModel(fmSimilarDbPath,0.8,999000)

<h4>Test Word2Vec model </h4>

In [11]:
s = fm.getSimilariityAll(fmSimilarDbPath,0.8)
model = gensim.models.Word2Vec.load(root + '/3_data/song2vec/model')
w2v_model_accuracy(model,s)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [ ]:
print ('Tiger & Dog', model.similarity('Tiger','Dog'))
print ('Tiger & Lion', model.similarity('Tiger','Lion'))
print ('Car & House',model.similarity('Car','House'))
print ('Lion & Cat',model.similarity('Lion','Cat'))
print ('Car & Van',model.similarity('Car','Van'))


### TSNE
#numOfDots = 20
#vocab = list(model.wv.vocab)  # Get back the tid list
#V = model[vocab]            # Get the word2vec vectors

#generateTSNE (V,vocab)
#s = fm.convertTrackIDTo(trackMetaDbPath,vocab,'title')
#generateTSNE (V,s)
